In [ ]:
# Build a model to predict the severity of an accident


In [1]:
import pandas as pd 

In [2]:
df = pd.read_csv('Data-Collisions.csv')

In [3]:
df.columns

Index(['SEVERITYCODE', 'X', 'Y', 'OBJECTID', 'INCKEY', 'COLDETKEY', 'REPORTNO',
       'STATUS', 'ADDRTYPE', 'INTKEY', 'LOCATION', 'EXCEPTRSNCODE',
       'EXCEPTRSNDESC', 'SEVERITYCODE.1', 'SEVERITYDESC', 'COLLISIONTYPE',
       'PERSONCOUNT', 'PEDCOUNT', 'PEDCYLCOUNT', 'VEHCOUNT', 'INCDATE',
       'INCDTTM', 'JUNCTIONTYPE', 'SDOT_COLCODE', 'SDOT_COLDESC',
       'INATTENTIONIND', 'UNDERINFL', 'WEATHER', 'ROADCOND', 'LIGHTCOND',
       'PEDROWNOTGRNT', 'SDOTCOLNUM', 'SPEEDING', 'ST_COLCODE', 'ST_COLDESC',
       'SEGLANEKEY', 'CROSSWALKKEY', 'HITPARKEDCAR'],
      dtype='object')

In [4]:
df['ROADCOND'].value_counts(dropna=False)

Dry               124510
Wet                47474
Unknown            15078
NaN                 5012
Ice                 1209
Snow/Slush          1004
Other                132
Standing Water       115
Sand/Mud/Dirt         75
Oil                   64
Name: ROADCOND, dtype: int64

In [5]:
df.head()

,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,...,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,...,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,...,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,...,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,...,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


So this data is interesting and all but.. I'm from Texas.

I want to look at Texas data.

While researching this, I found that these records do indeed exist.

https://data.world/spatialaustin/texas-crash-records-information-system-cris-extract

I downloaded the 'field lookup' file to see if "crash severity" was a field I could target.


In [26]:
df = pd.read_csv('lookup_2010_2016.csv', encoding = "ISO-8859-1")

In [27]:
df.head()

,ColumnName,ID,Description
0,AGENCY_ID,8,ABERNATHY POLICE DEPARTMENT
1,AGENCY_ID,9,ABILENE CHRISTIAN UNIVERSITY POLICE DEPARTMENT
2,AGENCY_ID,10,ABILENE POLICE DEPARTMENT
3,AGENCY_ID,12,ADDISON POLICE DEPARTMENT
4,AGENCY_ID,13,ALAMO COMMUNITY COLLEGE DISTRICT POLICE DEPART...


In [28]:
columns = df['ColumnName'].value_counts().to_frame()

In [29]:
columns = columns.index.values.tolist()
columns

['VEH_MOD_ID',
 'AGENCY_ID',
 'CITY_ID',
 'INV_DA_ID',
 'VEH_MAKE_ID',
 'CNTY_ID',
 'STATE_ID',
 'DRVR_LIC_RESTRIC_ID',
 'CONTRIB_FACTR_ID',
 'OTHR_FACTR_ID',
 'OBJ_STRUCK_ID',
 'COLLSN_ID',
 'CULVERT_TYPE_ID',
 'VEH_DMAG_AREA_ID',
 'BRIDGE_RTE_STRUCT_FUNC_ID',
 'BRIDGE_IR_STRUCT_FUNC_ID',
 'VEH_COLOR_ID',
 'MPO_ID',
 'CMV_EVENT_ID',
 'CHARGE_CAT_ID',
 'VEH_BODY_STYL_ID',
 'RWY_SYS_ID',
 'TRAFFIC_CNTL_ID',
 'INV_REGION_ID',
 'OCCPNT_POS_ID',
 'CMV_CARGO_BODY_ID',
 'PHYS_FEATR_ID',
 'CMV_VEH_TYPE_ID',
 'DRUG_CATEGORY_ID',
 'HARM_EVNT_ID',
 'REST_ID',
 'ROAD_CLS_ID',
 'VEH_DFCT_ID',
 'DRVR_LIC_ENDORS_ID',
 'WTHR_COND_ID',
 'DRVR_LIC_CLS_ID',
 'FUNC_SYS_ID',
 'POP_GROUP_ID',
 'WDCODE_ID',
 'VEH_TRVL_DIR_ID',
 'SURF_COND_ID',
 'PRSN_TYPE_ID',
 'VEH_DMAG_SCL_ID',
 'ENTR_ROAD_ID',
 'BRIDGE_SRVC_TYPE_ON_ID',
 'BRIDGE_SRVC_TYPE_UNDER_ID',
 'UNIT_DESC_ID',
 'BRIDGE_DETAIL_ID',
 'LIGHT_COND_ID',
 'ROAD_ALGN_ID',
 'AIRBAG_ID',
 'INS_TYPE_ID',
 'DRVR_LIC_TYPE_ID',
 'ROAD_PART_ID',
 'ETHNICITY_ID',

haha! CRASH_SEV_ID is indeed listed.

In [33]:
df.loc[df['ColumnName'] == 'CRASH_SEV_ID']

,ColumnName,ID,Description
5486,CRASH_SEV_ID,0,UNKNOWN
5487,CRASH_SEV_ID,1,INCAPACITATING INJURY
5488,CRASH_SEV_ID,2,NON-INCAPACITATING
5489,CRASH_SEV_ID,3,POSSIBLE INJURY
5490,CRASH_SEV_ID,4,FATAL
5491,CRASH_SEV_ID,5,NOT INJURED
5492,CRASH_SEV_ID,94,REPORTED INVALID
5493,CRASH_SEV_ID,95,NOT REPORTED


Accessing this data requires signing up for the CRIS database.

https://www.txdot.gov/government/enforcement/data-access.html

http://ftp.dot.state.tx.us/pub/txdot-info/trf/crash_statistics/automated/cris-guide.pdf

https://cris.dot.state.tx.us/secure/Share/app/home/welcome

You can only request one year of data at a time. I will be using data from the year 2019 (1/1-12/31) to build my model.


It took about a day to get the data from the CRIS team, but once I did, it came in an encrypted zip file. Inside of that were several more zip files, and judging by the filenames, it's the data broken up into smaller date ranges.

![title](img/1.png)

I unzipped the first archive, and poked around the data. The file 'extract_public_2018_20200825003448_crash_20190101-20190301Texas.csv' had the target data, CRASH_SEV_ID. 

My first task is going to be extracting all of the data and pulling it into a pandas dataframe.
